In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1) #input 30*30*1 OUtput? 28*28*8
        self.conv2 = nn.Conv2d(8, 8, 3, padding=1) #input 30*30*8 OUtput? 28*28*8
        self.pool1 = nn.MaxPool2d(2, 2) #input 28*28*8 OUtput? 14*14*8
        self.conv3 = nn.Conv2d(8, 16, 3, padding=1) #input 16*16*8 OUtput? 14*14*16
        self.conv4 = nn.Conv2d(16, 16, 3, padding=1) #input 16*16*16 OUtput? 14*14*16
        self.pool2 = nn.MaxPool2d(2, 2) #input 14*14*16 OUtput?7*7*16
        self.conv5 = nn.Conv2d(16, 32, 3) #input7*7*16 OUtput?5*5*32
        self.conv6 = nn.Conv2d(32, 10, 1)
        self.gap = nn.AvgPool2d(kernel_size = 5)#input5*5*32 OUtput?1*1*32
        
        


    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv5(x))
        x = self.conv6(x)
        x = self.gap(x)
        x = x.view(-1,10)
        return F.log_softmax(x)
        

In [ ]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
            Conv2d-2            [-1, 8, 28, 28]             584
         MaxPool2d-3            [-1, 8, 14, 14]               0
            Conv2d-4           [-1, 16, 14, 14]           1,168
            Conv2d-5           [-1, 16, 14, 14]           2,320
         MaxPool2d-6             [-1, 16, 7, 7]               0
            Conv2d-7             [-1, 32, 5, 5]           4,640
            Conv2d-8             [-1, 10, 5, 5]             330
         AvgPool2d-9             [-1, 10, 1, 1]               0
Total params: 9,122
Trainable params: 9,122
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.17
Params size (MB): 0.03
Estimated Total Size (MB): 0.21
-----------------------------------------------

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [ ]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.5173, Accuracy: 8390/10000 (83.90%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.1737, Accuracy: 9482/10000 (94.82%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0365, Accuracy: 9874/10000 (98.74%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0381, Accuracy: 9867/10000 (98.67%)



loss=0.02411394566297531 batch_id=468: 100%|██████████| 469/469 [00:10<00:00, 43.72it/s]



Test set: Average loss: 0.0329, Accuracy: 9881/10000 (98.81%)

